## Find the shared Account network among the following the stream seeded accounts
* how much overlap is there in the people followed by seed accounts? 

## Tweet Processing
This uses utility code stored in `tweetscape-experiments/python/tweet_processing`
This utility library will mold as we do more experiments.

#### install 
install commands (assumgin you start from teh current directory)
```
cd ../../tweet_processing
pip install -e .
```

#### Loading data 

Instantiate a StreamTweetProcessor object with the twarc client and your data_dir. 

The code for this is in the next cell

@andre: Make sure you point the data_dir varialbe to whatever path you have the unzipped file I sent you
The data dir is strucutred like this:
```
data/
    group_name/
        tweets.csv
        ref_tweets.csv
        following.csv
    group_name2/
        tweets.csv
        ref_tweets.csv
        following.csv
```

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from twarc import Twarc2, expansions
from minio import Minio
from tweet_processing import StreamTweetProcessor

load_dotenv()

twarc_client = Twarc2(
    consumer_key=os.environ["consumer_key"], 
    consumer_secret=os.environ["consumer_secret"],
    access_token=os.environ["access_token"], 
    access_token_secret=os.environ["access_token_secret"]
)

minio_client = Minio(
    # 'experiment-data-minio.fly.dev:9000',
    'localhost:9000',
    access_key=os.environ["MINIO_ROOT_USER"],
    secret_key=os.environ["MINIO_ROOT_PASSWORD"],
    secure=False
)

tp = StreamTweetProcessor(twarc_client=twarc_client, minio_client=minio_client, bucket="stream-seeding")

# group_name = "longevity-pranab"
group_name = "CA-Abundance-Economy"
group_name = "nicktorba,rhyslindmark,visakanv"

df_following, df_tweets, df_ref_tweets = tp.remote_read_seed_data(group_name) 


In [ ]:
## Get a set of usernames that each seed account follows
users_following = {}
for user in df_following["referencer.username"].unique().tolist():
    users_following[user] = set(df_following[df_following["referencer.username"] == user]["username"].tolist())
    
## index will be useful to access metadata (followers_count) about these accounts
df_following.set_index("username", inplace=True)


### Create a dataframe that will show the following_overlap for each of the accounts followed by at least 1 of the seed accounts. 
### We are most interested in the intersection among the following of those accounts
### If the accounts are picked well, they will likely have interesting intersection among their shared interest

following_overlap = {}

for stream_user, following in users_following.items():
    for followed_user in following: 
        following_overlap.setdefault(followed_user, []).append(stream_user)

df_data = []

for followed_user, stream_users in following_overlap.items():
    num_followers_of_followed = df_following.loc[followed_user]["public_metrics.followers_count"]
    if isinstance(num_followers_of_followed, pd.Series):
        num_followers_of_followed = num_followers_of_followed.iloc[0]
    df_data.append([followed_user, stream_users, len(stream_users), num_followers_of_followed, f"https://twitter.com/{followed_user}"])
    
overlap_df = pd.DataFrame(df_data, columns=["followed.username", "stream_users", "num_stream_users", "num_followers_of_followed", "profile_link"])
overlap_df["num_stream_users"].value_counts()

### Show Overlap
Here, we show users that are followed by multiple seed accounts, sorted by the number of followers of that accounts. 

Filtering out to the lower follower accounts are a signal that that person talks abou their shared interest, and is an easy way to avoid constant reccomendation of accounts like Elon

In [ ]:
NUM_SEED_ACCOUNTS = 3
overlap_df[overlap_df["num_stream_users"]==NUM_SEED_ACCOUNTS].sort_values("num_followers_of_followed")